# Exploring Seattle Building Permit Data in Python

Before we get started with any data, we need to make sure our environment has all the Python dependencies we need. In the cell below, we'll use the `pip install` command to install those dependencies. Typically, you might run this command directly from a command-line in a terminal application. You can get the same effect from within a Jupyter notebook by prepending an exaclamation point `!` to that command, which indicates to the interpreter that the command should be passed through to the shell.

In [ ]:
# NOTE: we attempted to install these dependencies on all lab machines;
# you should only need to execute this cell if you're running the notebook in Google Colab
!wget http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz
!tar -zxf spatialindex-src-1.8.5.tar.gz
!cd spatialindex-src-1.8.5
!cd spatialindex-src-1.8.5 && ./configure && make && make install
!ldconfig
!pip install pandas==0.24.2 statsmodels==0.9.0 rtree geopandas shapely sodapy

## Loading datasets into our notebook

We'll start with two datasets as the basis of our analysis. We'll load a city of Seattle permits dataset as well as a shapefile with polygons defined for each of the neighborhoods in the state of Washington. We download these datasets into pandas DataFrames using the open source sodapy Python Socrata module.

In [ ]:
import pandas as pd
from sodapy import Socrata

def download_dataset(domain, dataset_id):
    # for this exercise, we're not using an app token,
    # but you *should* sign-up and register for an app_token if you want to use the Socrata API
    client = Socrata(domain, app_token=None)
    offset = None
    data = []
    batch_size = 1000

    while True:
        records = client.get(dataset_id, offset=offset, limit=batch_size)
        data.extend(records)
        if len(records) < batch_size:
            break
        offset = offset + batch_size if (offset) else batch_size

    return pd.DataFrame.from_dict(data)

def download_permits_dataset():
    return seattle_permits_df if "seattle_permits_df" in globals() else download_dataset("data.seattle.gov", "k44w-2dcq")

def download_neighborhoods_dataset():
    return wa_neighborhoods_df if "wa_neighborhoods_df" in globals() else download_dataset("robo.demo.socrata.com", "smef-bsgr")

# load Seattle permits data
seattle_permits_df = download_permits_dataset()

# load shapefile data
wa_neighborhoods_df = download_neighborhoods_dataset()

In the next few cells we'll do some exploration of our datasets using the `len`, `head`, and `value_counts` functions. We'll start by getting a sense of how many rows are in each of our datasets with the `len` function.

In [ ]:
len(seattle_permits_df)

In [ ]:
len(wa_neighborhoods_df)

Now let's see have a peek at the first 5 rows in each of those dataset using the `head` method. You can optionally pass a parameter for the number of rows you want to print if 5 isn't enough.

In [ ]:
seattle_permits_df.head()

In [ ]:
wa_neighborhoods_df.head()

Printing our dataframes like this gives us a sense of what columns exist, as well as the frequently occurring values in the dataset. But there's an even better way to detrmine the top values for a particular column.

In [ ]:
seattle_permits_df["applieddate"].value_counts(dropna=False).head(10)

## Removing missing values

The value counts make it clear that a lot of the value in the "applieddate" column are missing or null. Since the crux of our analysis requires time and location data, we need to handle those missing values. There a variety of ways you can handle missing data, but removing incomplete rows is the simplest, so it's what we'll do here today. In the next cell, we'll remove null dates and null latitude and longitude columns. There are also a lot of columns in the permits dataset that we won't use in this analysis. So we'll also filter down our dataset to just the columns we're interested in to reduce the amount of extraneous information.

In [ ]:
seattle_permits_df = seattle_permits_df[seattle_permits_df["applieddate"].notnull()]
seattle_permits_df = seattle_permits_df[seattle_permits_df["latitude"].notnull()]
seattle_permits_df = seattle_permits_df[seattle_permits_df["longitude"].notnull()]
seattle_permits_df = seattle_permits_df[["applieddate", "latitude", "longitude"]].reset_index(drop=True)
seattle_permits_df.head(10)

## Visualizing place-based data with Socrata Maps

Loading and visualizing geographical data is relatively straight-forward in the Socrata UI. Before doing any additional work in Python, let's embed a map created in Socrata using its embed link. You'll notice that since the map is embedded within an IFrame (using iPython's IFrame class), that it's dynamic and we have preserved all the functionality from the map experience in the platform like changing the zoom, and searching for data within the map.

In [ ]:
from IPython.display import IFrame
IFrame('https://robo.demo.socrata.com/dataset/Seattle-Building-Permits-Point-Map/7unc-ff4h/embed?width=800&height=600', width=800, height=600)

The map above is a point map. Each of the rows from our permit dataset is rendered as a point on the map. Any observations about this visualization?

I think we can do better with a multi-layer choropleth map -- also created within the Socrata platform -- where we aggregate our points based on the neighborhoods that they fall within.

In [ ]:
from IPython.display import IFrame
IFrame("https://robo.demo.socrata.com/dataset/Seattle-Building-Permits-Choropleth/i6rm-ys8r/embed?width=800&height=600", width=800, height=600)

Any observations about this choropleth map?

## Prepping our geographic columns

Just like in our choropleth map, we'll try to group our data by neighborhood, and we'll try to improve on our map by looking at how our permit applications change over time.

Notice that the neighborhood shapefile we imported from Socrata contained boundaries for all neighborhoods in the state of Washington. (Can you use the the `value_counts` method to determine the top cities in the `wa_neighborhoods_df` dataset?)

In [ ]:
# Can you use the the value_counts method to determine the top cities in the wa_neighborhoods_df dataset?

Let's filter that down to the city of Seattle, since that's the focus of this analysis.

In [ ]:
seattle_neighborhoods_df = wa_neighborhoods_df[wa_neighborhoods_df["city"] == "Seattle"].reset_index(drop=True)
seattle_neighborhoods_df.head(10)

In [ ]:
# Can you use the `len` method to determine the number of rows in our new seattle_neighborhoods_df dataset?

One of the really powerful features of notebooks is that you can execute arbitrary code to transform your data. Since data wrangling is one of the fundamental tasks in any data analysis project, it's essential that we have tools to reshape and slice our data. This is one of the strengths of the pandas in Python.

We have already loaded the underlying neighborhood shapes, but let's make our coordinate and shape columns a little more useful by changing them to types that are a little easier to work with in Python. We'll make use of the geopandas and shapely Python modules. Why are we doing this? We want a single geographic column in each of our datasets, and in order to perform a join on those geographic columns, we need them to be respresented as types that our join function understands.

In the next cell, we'll add a new Polygon column to our shapefile dataset. We'll write a function that takes a shape that's encoded in the geojson format, and transform it into a Polygon column. We'll use the `apply` method to do this transformation.

In [ ]:
from shapely.geometry import Polygon

def polygon_from_geojson(geojson):
    return Polygon(geojson["coordinates"][0][0])

seattle_neighborhoods_df["polygon"] = seattle_neighborhoods_df["the_geom"].apply(polygon_from_geojson)
seattle_neighborhoods_df["polygon"].head()

Next, we'll add a new Point column to our permits dataset. We write a function that takes an entire row from our dataset as an input, pulls out our latitude and longitude from that row, and returns a Point corresponding to those coordinates. Once again, we'll use the `apply` method to do this transformation.

In [ ]:
from shapely.geometry import Point

# create a new point column from latitude and longitude columns in permits dataset
def point_from_coordinates(row):
    coordinates = (row["longitude"], row["latitude"])
    return Point(coordinates)

seattle_permits_df["latitude"] = pd.to_numeric(seattle_permits_df["latitude"])
seattle_permits_df["longitude"] = pd.to_numeric(seattle_permits_df["longitude"])
seattle_permits_df["point"] = seattle_permits_df.apply(point_from_coordinates, axis=1)
seattle_permits_df.head()

## Performing a spatial join

Now that we our landuse dataframe and our neighborhoods dataframe, each with geometry columns, that next thing we want to do is perform a spatial join so we can have a single dataframe tying permits to neighborhoods. This will allow us to compare across neighborhoods, just as we did with the heat map earlier.

In [ ]:
import geopandas as gpd
permits_geo_df = gpd.GeoDataFrame(seattle_permits_df, geometry='point')
neighborhoods_geo_df = gpd.GeoDataFrame(seattle_neighborhoods_df, geometry='polygon')

joined_df = pd.DataFrame(gpd.sjoin(permits_geo_df, neighborhoods_geo_df, how="inner", op="within", rsuffix="neighborhood_"))
joined_df = joined_df[["applieddate", "name"]]#.sort_values("applieddate")  # ROBERT TODO: can I remove the sort?

## TODO: print the first rows of the joined data frame `joined_df`
joined_df.head()

## Aggregating based on neighborhood

Neat! Now that we have a single dataset with each row corresponding to a permit and its corresponding neighborhood, we can group our dataset by neighborhood, which is the first step to exploring the number of permits by neighborhood. To make our neighborhood-based analysis a little more digestible, we'll restrict it to a subset of Seattle neighborhoods.

In [ ]:
neighborhoods_of_interest = set(["Ballard", "Capitol Hill", "Magnolia", "South Lake Union", "North Beacon Hill", "Pioneer Square"])

def is_neighborhood_of_interest(neighborhood):
    return neighborhood in neighborhoods_of_interest

subset_df = joined_df[joined_df["name"].apply(is_neighborhood_of_interest)].reset_index(drop=True)
subset_df.groupby("name").count()

## Aggregating based on neighborhood and date

Having grouped our data based on neighborhood, the next thing we want to do is look at one aspect of the data that was missing from the choropleth map -- time. Just as we did with the `Point` and `Polygon` columns previously, we need to convert the `applieddate` column type in the permits dataframe to a `datetime` in order to benefit from some time-based functionality in Python. We'll also filter out dates before 2005 and during 2019 so the years are comparable. We also make the new `datetime` column the index column for our dataframe, which gives us some added benefits such as automatic sorting based on date and good default behavior when plotting our time series.

In [ ]:
import datetime

def is_between_2005_and_2018(date):
    return date > datetime.datetime(2005, 1, 1) and date < datetime.datetime(2019, 1, 1)

fixed_dates_df = subset_df.copy()
fixed_dates_df["applieddate"] = subset_df["applieddate"].apply(pd.to_datetime)
fixed_dates_df = fixed_dates_df[fixed_dates_df["applieddate"].apply(is_between_2005_and_2018)]
fixed_dates_df = fixed_dates_df.set_index(fixed_dates_df["applieddate"]).drop(["applieddate"], axis=1)
grouped = fixed_dates_df.groupby("name").resample("M").count()
grouped.head(10)

## Plotting a time series

Finally, with our data grouped by neighborhood and by month, let's visualize it! The code below is somewhat complex, so we won't go into it in detail. But the gist is this: we iterate over our groups (neighborhoods) and create a time series plot for each, labeling each plot based on its name, and coloring our line purple just because who doesn't like purple?

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use('ggplot')

ncols=2
nrows=3

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15,15))

for (name, neighborhood_df), axis in zip(grouped.groupby(level=0), axes.flatten()):
    axis.xaxis.label.set_visible(False)
    axis.set_ylim(0, 75)
    axis.set_xlim(left=datetime.date(2000, 1, 1), right=datetime.date(2018, 12, 31))
    axis.yaxis.set_major_formatter(FormatStrFormatter('%g'))
    neighborhood_df.xs(name).plot(title=name, style="", ax=axis, legend=False, color="purple")


What are your observations based on these plots?

## Understanding trend and seasonality

Let's select one of these neighboorhoods to drill in on. I'm partial to Ballard.

The statsmodels package in Python gives us a really useful set of tools for analyzing time series data. In particular, we can use the `seasonal_decompose` method to breakdown a time series into consituent series corresponding to the longterm trend in the data, the seasonal aspect of the data, and noise.

In [ ]:
import statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose

name, neighborhood_df = list(grouped.groupby(level=0))[0] # Ballard
result = seasonal_decompose(neighborhood_df.xs(name))
fig = result.plot()